## データ定義

In [174]:
import json
from requests_oauthlib import OAuth1Session
import re
import decimal
from multiprocessing import Process
from requests_oauthlib import OAuth1Session
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
import urllib.request, urllib.error
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options
import datetime

import sys
sys.path.append('../config')
import config

today = datetime.datetime.now().strftime('%Y%m%d')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome(chrome_options=chrome_options)
options = Options()
options.set_headless(True)

from pytrends.request import TrendReq
pytrends = TrendReq()

import MeCab
mecabTagger = MeCab.Tagger("-Ochasen")

# consumer_key = 'EGX737i2FcgfKGofWxOSAp6Wv'
# consumer_secret = 'We4QzsL0CBOsC33nSDgfMe6HiJCyeGqgiP1bx8IgZAxXSNWvQA'
# access_token = '979914304290893824-Kz3BsGhkr5oqNb2KiQkPUpSMnWI3HSR'
# access_token_secret = 'YqQQeXr7N5LgHa3KM3KxfO8llzQCtME5H9kBSa2xUwgcx'
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = 'YqQQeXr7N5LgHa3KM3KxfO8llzQCtME5H9kBSa2xUwgcx'


twitter = OAuth1Session(consumer_key, consumer_secret, access_token, access_token_secret)
url = "https://api.twitter.com/1.1/trends/place.json"

def get(url, params=None):
    response = twitter.get(url, params=params)
    text = response.text.replace(':""', ':null') 
    results = pd.read_json(text)
    trends = pd.DataFrame(results['trends'].values[0])
    trends['location']=results['locations'][0][0]['name']
    trends['woeid']=results['locations'][0][0]['woeid']
    return trends

/usr/bin/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: use options instead of chrome_options
/usr/bin/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: use setter for headless property instead of set_headless


## 辞書取得

### 食材リストの取得

In [153]:
oisix_words = pd.read_csv('./input/食材辞書.csv',encoding = 'shift_jis')

### ロケーションの取得

In [154]:
locations = pd.read_csv('./input/locations.csv')

## トレンド取得

### ツイッタートレンドの取得

In [155]:
trends = []

for w in locations['WOEID'][21:]:
    if len(trends) > 0:
        twitter_rank = trends.append(get("https://api.twitter.com/1.1/trends/place.json", params={"id": w}))
    else:
        twitter_rank = get("https://api.twitter.com/1.1/trends/place.json", params={"id": w})

### Googleトレンドの取得

In [166]:
url = "https://trends.google.co.jp/trends/trendingsearches/daily?geo=JP"
driver.get(url)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

def get_ggl_rank(soup):
    rank_list = []
    url_list = []
    for i in range (20):
        try:
            text = re.sub('\n','',soup.select_one("body > div.trends-wrapper > div:nth-of-type(2) > div > div.feed-content > div > div.generic-container-wrapper > ng-include > div > div > div > div:nth-of-type(1) > \
                md-list:nth-of-type({}) > feed-item > ng-include > div > div > div.details-wrapper > div.details > div.details-top > div > span > a".format(i)).text).strip()
            rank_list = rank_list + [text]
            url = soup.select_one('#feed-item-{} > div.details-wrapper > div.details > div.details-bottom > div.subtitles-text-wrapper.visible > div.summary-text > a'.format(text)).attrs['ng-href']
            url_list = url_list + [url]

        except:
            continue
    return rank_list,url_list

ggl_rank,ggl_url = get_ggl_rank(soup)

### BigLobeトレンドの取得

In [165]:
url = "https://search.biglobe.ne.jp/ranking/"
driver.get(url)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

biglobe_rank = [soup.select_one('#word-rankBOX-d > div:nth-of-type({}) > div > p.word-rank_rankWORD1-5 > a'.format(i)).text for i in range(2,12)]

### Gooトレンドの取得

In [169]:
# url = "https://ranking.goo.ne.jp/ranking/282/"
# driver.get(url)
# html = driver.page_source.encode('utf-8')
# soup = BeautifulSoup(html, "html.parser")

# goo_rank = [re.sub('\n|\t','',soup.select_one('#NR-main-in > div.list-items.no-border > dl > dd:nth-of-type(1) > ul > li:nth-of-type({}) > div.ranking-content > div > div > h2'.format(i)).text) for i in [1,2,3,5,6,7,9,10,11,12]]

### ついップルトレンドの取得

In [172]:
url = "https://tr.twipple.jp/hotword/"
driver.get(url)
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

twiple_rank = [soup.select_one('#rankArea > div:nth-child({}) > div.lBox > div.rankTtl > a'.format(i)).text for i in range(1,11)]

In [173]:
twiple_rank

['土砂降り',
 '#ij954',
 'テロ',
 'MAX',
 'アマナ',
 '#ノンストップ',
 '#ウエルシア',
 'Apex',
 '属性の神',
 '市役所']

### トレンドのマージ

In [67]:
today

'20190424'

In [72]:
#Twitterのランキング
twitter_df = pd.DataFrame({'date' : [today]* len(twitter_rank), 'source' : ['Twitter']* len(twitter_rank), 'name' : twitter_rank['name']})

#Googleのランキング
google_df = pd.DataFrame({'date' : [today]* len(ggl_rank),'source' : ['Google']* len(ggl_rank), 'name' :  ggl_rank})

#Biglobeのランキング
biglobe_df = pd.DataFrame({'date' : [today]* len(biglobe_rank),'source' : ['Biglobe']* len(biglobe_rank), 'name' :  biglobe_rank})

#Gooのランキング
goo_df = pd.DataFrame({'date' : [today]* len(goo_rank),'source' : ['Goo']* len(goo_rank), 'name' :  goo_rank})

#Twipleのランキング
twiple_df = pd.DataFrame({'date' : [today]* len(twiple_rank),'source' : ['Twiple']* len(twiple_rank), 'name' :  twiple_rank})

In [73]:
rank_df = pd.concat([twitter_df,google_df,biglobe_df,goo_df,twiple_df],axis=0)

### トレンドデータを仮保存

In [49]:
rank_df.reset_index(drop=True).to_feather('./output/trend_{}.ftr'.format(today))

In [83]:
rank_df = pd.read_feather('./output/trend_{}.ftr'.format(today))

## オイシックス関連ワードの分別

### 食材リストによる辞書判定

In [84]:
rank_df['oisix'] = rank_df['name'].apply(lambda x : sum([ow in x for ow in oisix_words['食材詳細']])>0)

### google分類による判定（mecabによる名詞の抽出→タイプ分析）

In [85]:
n_list = []
for text in rank_df['name']:
    node = mecabTagger.parse(text)
    df = pd.DataFrame([x.split("\t") for x in node.split("\n")])
    nauns = df[df[3].fillna('').str.contains('名詞')][0].tolist()
    for w in nauns:
        n_list.extend([[text, w]])    
        
df = pd.DataFrame(n_list,columns = ['word','wakachi'])        

In [86]:
# プログレスバーのラベル設定
tqdm_notebook.pandas(desc="hoge progress: ")

def ggl_categorize(x):
    try:
        return  [y['type'] for y in pytrends.suggestions(x)][:3]
    except:
        return  []

df['category'] = df['wakachi'].progress_apply(lambda x :ggl_categorize(x))

In [89]:
keyword = ['dessert','food', 'plants' ,'tea' , 'dish' ,'noodle','dough','alcohol','algae','amphibians',
          'animal','appetizer','aacteria','bagel','bakery','barbecue','bean','beef','beer','beverage','bird','bivalve','bread','breakfast','cafe','cake','cheese',
          'fungus','seed','konjac','restaurant','fruit','berry','cookie','fish','pasta','confectionery','condiment','grape','surimi','tofu','cook','flour','spice','drink',
          'rice','meat','chicken','pork','vegetable','juice','sprout','tsukemono','poteto','grape','sauce','wine','milk','sweet','yogurt']

df['flg'] = df['category'].apply(lambda x : 1 if  sum([key in y.lower() for key in keyword for y in x])>0 else 0 )

In [101]:
result = pd.merge(rank_df, pd.DataFrame(df.groupby('word')['flg2'].apply(lambda x : sum(x)>0))['flg2'],left_on='name', right_on='word')

### mecabで分解してからの辞書一致

In [91]:
# プログレスバーのラベル設定
tqdm_notebook.pandas(desc="hoge progress: ")
df['flg2'] = df['wakachi'].progress_apply(lambda x : sum(oisix_words['食材詳細'] == x)>0)

In [93]:
key in y.lower() for key in keyword for y in x

SyntaxError: invalid syntax (<ipython-input-93-81f7cc87421c>, line 1)

In [102]:
df[df['flg2']>0]

,word,wakachi,category,flg,flg2
71,ヤギのチーズ,チーズ,"[Dairy product, Dessert, Family entertainment ...",1,True
207,#nhk_suppin,_,[Studio album by Caravan Palace],0,True


In [104]:
result[result['oisix']|result['flg2']]

,source,name,date,oisix,flg2
2,Twitter,#平成のあなたが秘密にしていたこと,20190424,True,False
26,Twitter,ヤギのチーズ,20190424,True,True
64,Biglobe,和食,20190424,True,False
81,Twiple,#nhk_suppin,20190424,True,True
